# MongoDB Atlas Search: Basic Search

## Setup

In [ ]:
# import our libraries, instantiate our classes

import sys

from pymongo import MongoClient
import pprint

client = MongoClient('')['']

pp = pprint.PrettyPrinter(indent=1)

### Example profile
anneli.al.larsson63885@mendoza-dickson.net   
anneli.al.larsson73885@mendoza-dickson.net

## Search index

```json
{
    "mappings": {
        "dynamic": false,
        "fields": {
            "contactMethods": {
                "fields": {
                    "value": {
                        "analyzer": "regReplace",
                        "multi": {
                            "keywordAnalyzer": {
                                "analyzer": "lucene.keyword",
                                "type": "string"
                            }
                        },
                        "type": "string"
                    }
                },
                "type": "document"
            }
        }
    },
    "analyzers": [
        {
            "charFilters": [],
            "name": "regReplace",
            "tokenFilters": [
                {
                    "type": "lowercase"
                },
                {
                    "matches": "all",
                    "pattern": "[^0-9]?",
                    "replacement": "-",
                    "type": "regex"
                }
            ],
            "tokenizer": {
                "type": "keyword"
            }
        }
    ]
}
```

### Fuzzy without numbers

In [ ]:
emailToSearch = 'ingridkarlsson79050@gordon.com'
search = [
    {
        '$search': {
            'index': 'regexNumber', 
            'compound': {
                'must': [
                    {
                        'text': {
                            'query': emailToSearch, 
                            'path': {
                                'value': 'contactMethods.value', 
                                'multi': 'keywordAnalyzer'
                            }, 
                            'fuzzy': {
                                'maxEdits': 1
                            }
                        }
                    }
                ], 
                'should': [
                    {
                        'text': {
                            'query': emailToSearch, 
                            'path': 'contactMethods.value', 
                            'score': {
                                'boost': {
                                    'value': 1.1
                                }
                            }
                        }
                    }, {
                        'text': {
                            'query': emailToSearch, 
                            'path': {
                                'value': 'contactMethods.value', 
                                'multi': 'keywordAnalyzer'
                            }
                        }
                    }
                ]
            }, 
            'highlight': {
                'path': 'contactMethods.value'
            }
        }
    }, {
        '$limit': 5
    }, {
        '$project': {
            'contactMethods': 1, 
            'score': 1
        }
    }, {
        '$unwind': {
            'path': '$contactMethods'
        }
    }, {
        '$match': {
            'contactMethods.type': 'EMAIL'
        }
    }, {
        '$replaceRoot': {
            'newRoot': '$contactMethods'
        }
    }, {
        '$addFields': {
            'score': {
                '$meta': 'searchScore'
            }, 
            'highlights': {
                '$meta': 'searchHighlights'
            }
        }
    }, {
        '$project': {
            'value': 1, 
            'score': 1, 
            'highlights': 1
        }
    }, {
        '$sort': {
            'score': -1
        }
    }
]

result = list(client['profiles'].aggregate(search))
pp.pprint(result)

### Find similar profiles - Let's fetch an example document

In [26]:
# fetch a sample document, in this case, the movie "The Godfather"
one_document_query = [
    {'$match':{"partyUId":"acb53c20-f2b1-49ce-9023-90b86b7bbc55"}},
    {'$project':{
        "partyUId" : 1,
        "firstName":1,
        "lastName":1,
        "contactMethods.type" : 1,
        "contactMethods.value" : 1,
        "_id":0
    }},
    {"$limit":1}
]

result = list(client['profiles'].aggregate(one_document_query))
pp.pprint(result)

[{'contactMethods': [{'type': 'SMS', 'value': '058-45 19 75'},
                     {'type': 'EMAIL',
                      'value': 'astquicklyridlarsson@andersson.org'}],
  'firstName': 'Astrid',
  'lastName': 'Larsson',
  'partyUId': 'acb53c20-f2b1-49ce-9023-90b86b7bbc55'}]


#### Now, we provide the result above as a reference document to the MoreLikeThis query:

In [27]:
# Construct 
more_like_this_query = [
    {
        '$search': {
            'index': 'moreLikeThis', 
            'moreLikeThis': {
                'like': result
            }
        }
    },
    {
        '$project':{
            "partyUId" : 1,
            "firstName":1,
            "lastName":1,
            "contactMethods.type" : 1,
            "contactMethods.value" : 1,
            "_id":0,
            'score': {
                '$meta': 'searchScore'
            },
            "highlights": { "$meta": "searchHighlights" }
        }
    },
    {'$limit':2}
]

pp.pprint(list(client['profiles'].aggregate(more_like_this_query)))

[{'contactMethods': [{'type': 'SMS', 'value': '058-45 19 75'},
                     {'type': 'EMAIL',
                      'value': 'astquicklyridlarsson@andersson.org'}],
  'firstName': 'Astrid',
  'lastName': 'Larsson',
  'partyUId': 'acb53c20-f2b1-49ce-9023-90b86b7bbc55',
  'score': 16.047607421875},
 {'contactMethods': [{'type': 'SMS', 'value': '058-45 19 75'},
                     {'type': 'EMAIL',
                      'value': 'astrid.larsson2470@green.com'}],
  'firstName': 'Astrid',
  'lastName': 'Larsson',
  'partyUId': 'acb53c20-f2b1-49ce-9022-90b86b7bbc55',
  'score': 12.758387565612793}]
